# **Model design and testing**
*This script attempts different model designs for a grey matter morphology mult-compartment model and tests all versions of the model on an area within the left hippocampus. The result of the script is two saved json files, one containing MSE values for the different models and one containing R2 values. Because for some reason it does not work to loop over multiple subjects. therefore the resulting dataframes only contain one subject and has to be run multiple times to get data from several subjects (merged later).*

In [ ]:
# import packages
import numpy as np
import nibabel as nib
from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from dmipy.signal_models import cylinder_models, sphere_models, gaussian_models
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models import cylinder_models, gaussian_models, sphere_models
from dmipy.distributions import distribute_models
import pandas as pd
from scipy.ndimage import binary_erosion

### **Defining Functions**
*A few functions for necessary tasks were pre-defined for improved readability of code*

- *Function for loading a nifti file (load_nifti)*
    - *This function takes a filename as an argument, loads a nifti file by this name, extracts the data matrix and returns it*
- *Function for extracting the mask of one structure from the mask file (extract_single_mask)*
    - *This function takes two arguments. The first argument is the mask file, containing the segmented mask of every segmented structure. The second argument, the "mask_index" is the labels of the desired subcortical structure. The function proceeds to copy the original mask file (as to not interfer with the original data), collect the correct structure, make the mask binary and perfom erosion before returning the eroded binary mask. (there are a different amount of voxels in images because of enabled interpolation in beginning of data acquisition. To get aproximately the same amount of data points for all models the iterations during erosion is changed manually (2 for first half and 1 for second half)*
- *Function for extracting all values from the DWI file belonging to mask (extract_values)*
    - *This function takes two arguments, the full DWI data matrix and a mask (result from extract_single_mask). The function returns a 3-dimensional that only contains the voxels covered by the maks*

In [ ]:
def load_nifti(filename):
    nifti_file = nib.load(filename)
    data = nifti_file.get_fdata()
    return data

def extract_single_mask(all_masks_file, mask_index): # mask_index is value of structure
    # Create mask for left hippocampus and make it binary
    mask = all_masks_file.copy()
    mask[mask != mask_index]  = 0  #all values not the right label to 0
    mask = (mask != 0).astype(np.uint8) #make it binary
    mask = binary_erosion(mask, iterations=2).astype(mask.dtype) # perform erosion to get rid of edge values (iterations 2 for 001-009 og 1 for 010-017)
    return mask

def extract_values(dwi_data, mask):
    nonzero = np.where(mask > 0)
    values_array = dwi_data[nonzero] 
    return values_array

### **Set up acquisition scheme**
*Seperate files containing the b-values, b-vectors and small delta ($\delta$) and big delta ($\Delta$) values used during acquisition is loaded to be used for setting up the acquisition scheme. All values in these files are ordered the same as the DWI shells in the merged data file and after loading they are converted to SI units*

In [ ]:
# set up acquisition scheme
bval = np.loadtxt('all.bval') * 1e6 # load and convert from [s/mm²] to SI unit [s/m²]
bvec = np.transpose(np.loadtxt('all.bvec'))
delta = np.loadtxt('small_delta.txt')*10e-4 #load and convert from [ms] to SI unit [s] 
Delta = np.loadtxt('big_delta.txt')*10e-4 #load and convert from [ms] to SI unit [s] 

acq_scheme = acquisition_scheme_from_bvalues(bval, bvec, delta, Delta)

### **Prepare basis for initializing different models**
*All model designs will use many of the same base compartments. These compartments are defined below. There are spheres for modelling cells with restricted diffusion (diffusivity set at 1.0e-09 m²/s), a ball for free diffusion, sticks for restricted diffusion along axons and and microglial processes (diffusivity set at 1.0e-09 m²/s) and a zeppelin for extracellular diffusion. The stick and zeppelin is combined in a watson dispersed bundle.*

In [ ]:
# Base for multi compartment models
small_sphere = sphere_models.S4SphereGaussianPhaseApproximation(diffusion_constant=1.0e-9)#, diameter= 4.0e-6)
big_sphere = sphere_models.S4SphereGaussianPhaseApproximation(diffusion_constant=1.0e-9)#, diameter= 1.0e-5)
free_water = gaussian_models.G1Ball()
stick = cylinder_models.C1Stick()
zeppelin = gaussian_models.G2Zeppelin()
watson_dispersed_bundle = distribute_models.SD1WatsonDistributed(models=[stick, zeppelin]) 
watson_dispersed_bundle.set_fixed_parameter('C1Stick_1_lambda_par', 1.0e-9)

### **Designing and fitting different models**
*In the following cell the DWI data file along with the mask of one subject is loaded. A mask of only the hippocampus is then created from the mask file and used to select only the voxels of the hippocampus from the DWI datafile.*

*After this six differet models are initialized and fitted to the data. These model are described in short here:*
- *Model 1: In this model the ranges for the diamater of the spheres are set to 1-10 $\mu m$ for the small sphere and 10-30 $\mu m$*
- *Model 2: In this model the ranges for the diamater of the spheres are set to 6-8 $\mu m$ for the small sphere and 18-22 $\mu m$*
- *Model 3: This model only uses one sphere. The ranges for the diameter of this sphere is 1-30 $\mu m$*
- *Model 4: In this model the ranges for the diamater of the spheres are set to 1-20 $\mu m$ for the small sphere and 100-200 $\mu m$*
- *Model 5: In this model the ranges for the diamater of the spheres are set to 1-18 $\mu m$ for the small sphere and 18-25 $\mu m$*
- *Model 6: In this model the ranges for the diamater of the spheres are set to 1-10 $\mu m$ for the small sphere and 10-30 $\mu m$ (same as Model 1), and a constraint is also put on the volume fraction of the ball compartment limiting it to the range 0-0.25*

*Note: The diffusivity of the ball is set to 3.0e-09 m²/s for all models*

In [ ]:
ID = '' # select file

file = ID + '.nii.gz'
dwi_data = load_nifti('pre_processing/4_normalized_for_TE/' + file)
mask_data = load_nifti('pre_processing/5_segmentation/masks/'+ file)

hippocampus_mask = extract_single_mask(mask_data, 17) # 17 is label of left hippocampus
ROI = extract_values(dwi_data, hippocampus_mask)

microglia_model_1 = MultiCompartmentModel(models=(small_sphere, big_sphere, free_water, watson_dispersed_bundle))
microglia_model_1.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[1.0e-6, 1.0e-5])
microglia_model_1.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_2_diameter',[1.0e-5, 3.0e-5])
microglia_model_1.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09)
fitted_model_1 = microglia_model_1.fit(acq_scheme, ROI , solver='mix')

microglia_model_2 = MultiCompartmentModel(models=(small_sphere, big_sphere, free_water, watson_dispersed_bundle))
microglia_model_2.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[6.0e-6, 8.0e-6])
microglia_model_2.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_2_diameter',[1.8e-5, 2.2e-5])
microglia_model_2.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09)
fitted_model_2 = microglia_model_2.fit(acq_scheme, ROI, solver='mix')

microglia_model_3 = MultiCompartmentModel(models=(small_sphere, free_water, watson_dispersed_bundle))
microglia_model_3.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[1.0e-6, 3.0e-5])
microglia_model_3.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09)
fitted_model_3 = microglia_model_3.fit(acq_scheme, ROI, solver='mix')

microglia_model_4 = MultiCompartmentModel(models=(small_sphere, big_sphere, free_water, watson_dispersed_bundle))
microglia_model_4.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[1.0e-6, 2.0e-5])
microglia_model_4.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_2_diameter',[1.0e-4, 2.0e-4]) 
microglia_model_4.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09)
fitted_model_4 = microglia_model_4.fit(acq_scheme, ROI, solver='mix')

microglia_model_5 = MultiCompartmentModel(models=(small_sphere, big_sphere, free_water, watson_dispersed_bundle))
microglia_model_5.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[1.0e-6, 1.8e-5])
microglia_model_5.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_2_diameter',[1.8e-5, 2.5e-5]) 
microglia_model_5.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09)
fitted_model_5 = microglia_model_5.fit(acq_scheme, ROI, solver='mix')

microglia_model_6 = MultiCompartmentModel(models=(small_sphere, big_sphere, free_water, watson_dispersed_bundle))
microglia_model_6.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[1.0e-6, 1.0e-5])
microglia_model_6.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_2_diameter',[1.0e-5, 3.0e-5])
microglia_model_6.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09)
microglia_model_6.set_parameter_optimization_bounds('partial_volume_2',[0, 0.25])
fitted_model_6 = microglia_model_6.fit(acq_scheme, ROI, solver='mix')


### **Saving MSE and R2 to dataframes**
*In the process of fitting the model Dmipy calculates both the mean squared error (MSE) and the R2 coefficient of determination. Below is a snippet of code made to store these values into a dataframe for saving. Two dataframes are initialized with the model names as column names. As the code will not loop, the dataframes will be saved with only one row, but the ID is still used as the row index. Arrays containing MSE and R2 values are then stored in the cells of the dataframe and the dataframe is saved as a json file*

In [ ]:
model_names = ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5', 'Model 6'] 
fitted_models = [fitted_model_1, fitted_model_2, fitted_model_3, fitted_model_4, fitted_model_5, fitted_model_6] 
MSE = pd.DataFrame(index=[ID] , columns=[i for i in model_names])
R2 = pd.DataFrame(index=[ID] , columns=[i for i in model_names])

for idx, model in enumerate(model_names):
    model_number = idx+1
    MSE.loc[ID, model] = [fitted_models[idx].mean_squared_error(ROI)]
    R2.loc[ID, model] = [fitted_models[idx].R2_coefficient_of_determination(ROI)]

MSE.to_json('MSE_dataframes/MSE_scores_'+ID+'.json', orient='index')
R2.to_json('R2_dataframes/R2_scores_'+ID+'.json', orient='index')